In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

def deteksi_helm_tanpa_xml_enhanced(img_rgb_input):
    
    img_resized = cv2.resize(img_rgb_input, (640, 640))
    img_height, img_width, _ = img_resized.shape
    
    img_blurred = cv2.GaussianBlur(img_resized, (5, 5), 0)
    
    img_ycrcb = cv2.cvtColor(img_blurred, cv2.COLOR_RGB2YCrCb)
    
    lower_skin = np.array([80, 133, 77], dtype=np.uint8)
    upper_skin = np.array([255, 173, 127], dtype=np.uint8)
    
    mask_skin = cv2.inRange(img_ycrcb, lower_skin, upper_skin)
    
    kernel_open = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    mask_cleaned = cv2.morphologyEx(mask_skin, cv2.MORPH_OPEN, kernel_open)
    
    kernel_close = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (20, 20))
    mask_cleaned = cv2.morphologyEx(mask_cleaned, cv2.MORPH_CLOSE, kernel_close)
    
    contours, _ = cv2.findContours(mask_cleaned, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    ada_wajah_terbuka = False
    img_hasil = img_resized.copy()
    
    contours = sorted(contours, key=cv2.contourArea, reverse=True)
    
    for cnt in contours:
        area = cv2.contourArea(cnt)
        x, y, w, h = cv2.boundingRect(cnt)
        aspect_ratio = w / h if h > 0 else 0
        
        hull = cv2.convexHull(cnt)
        hull_area = cv2.contourArea(hull)
        solidity = float(area) / hull_area if hull_area > 0 else 0
        
        is_large_enough = area > 3000
        is_top_position = y < img_height * 0.6
        is_shape_face = 0.4 < aspect_ratio < 1.2
        is_solid = solidity > 0.6
        
        if is_large_enough and is_top_position and is_shape_face and is_solid:
            ada_wajah_terbuka = True
            
            cv2.rectangle(img_hasil, (x, y), (x+w, y+h), (255, 0, 0), 3)
            cv2.putText(img_hasil, "Wajah", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,0,0), 2)
            break
            
    if ada_wajah_terbuka:
        keputusan = "Tidak Pakai Helm"
        warna_status = (255, 0, 0)
    else:
        keputusan = "Pakai Helm / Aman"
        warna_status = (0, 255, 0)
        
    cv2.putText(img_hasil, keputusan, (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.2, warna_status, 3)
    
    return img_hasil, mask_cleaned, keputusan

uploader = widgets.FileUpload(accept='image/*', description='Upload Gambar', multiple=False)
process_button = widgets.Button(description='Proses Gambar', button_style='success', icon='check')
output_display = widgets.Output()

def on_process_button_clicked(b):
    with output_display:
        clear_output(wait=True)
        
        if not uploader.value:
            print("Silakan upload file gambar terlebih dahulu.")
            return

        try:
            if isinstance(uploader.value, tuple):
                uploaded_file = uploader.value[0]
            elif isinstance(uploader.value, dict):
                uploaded_file = list(uploader.value.values())[0]
            else:
                uploaded_file = uploader.value[0]
            file_content = uploaded_file['content']
        except Exception as e:
            print(f"Error data: {e}")
            uploader.value.clear()
            return

        nparr = np.frombuffer(file_content, np.uint8)
        img_bgr = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
        
        if img_bgr is None:
            print("Gagal membaca gambar.")
            return

        img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
        
        gambar_hasil, mask_hasil, hasil_teks = deteksi_helm_tanpa_xml_enhanced(img_rgb)
        
        print(f"Status: {hasil_teks}")

        plt.figure(figsize=(14, 7))
        
        plt.subplot(1, 2, 1)
        plt.imshow(mask_hasil, cmap='gray')
        plt.title("Segmentasi Kulit (YCrCb)")
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.imshow(gambar_hasil)
        plt.title("Hasil Deteksi")
        plt.axis('off')
        plt.show()
        
        uploader.value.clear()

process_button.on_click(on_process_button_clicked)
display(uploader, process_button, output_display)